In [1]:
library(data.table)

In [2]:
#load central functions
central_funks = list.files('/home/j/temp/central_comp/libraries/current/r/', ".R")

In [3]:
for(fff in central_funks) {
    source(paste0('/home/j/temp/central_comp/libraries/current/r/',fff))
}

In [4]:
#get the ages for relative risks
demos = get_demographics(gbd_team = 'epi')

In [5]:
#load the relative risk sheets
rrs = data.table(read.csv('/home/j/temp/dccasey/temperature/risk/placeholder_bunker_temperature_risk.csv'))

In [6]:
#cross over by morbid/mort
cross_over = data.table(table(rrs[,c('acause','measure')]))

In [7]:
#reshape wide
cross_over = dcast(cross_over, acause~measure) #will need to be 

Using 'N' as value column. Use 'value.var' to override


In [8]:
#get causes with only mort or morbid
miss_morbid = unique(cross_over[morbidity == 0 ,acause])
add_morbid = rrs[acause %in% miss_morbid,]
miss_mort = unique(cross_over[mortality == 0 ,acause])
add_mort = rrs[acause %in% miss_mort,]

In [9]:
#change the measure
add_morbid = add_morbid[,measure := 'morbidity']
add_mort = add_mort[,measure := 'mortality']

#combine
rrs = rbindlist(list(rrs, add_morbid, add_mort))

In [10]:
#make a risk grid
risk_grid = expand.grid(age_group_id = demos$age_group_ids, sex_id = 3, acause = unique(rrs[,acause]))

In [11]:
#merge in the rrs
risk_grid = data.table(merge(risk_grid, rrs, by = 'acause'))

In [12]:
#create draws
#back calculate SD
risk_grid = risk_grid[, sd := ((upper-mean)/1.96)]
setnames(risk_grid, c('mean', 'upper', 'lower','sd'), c('pc_mean','pc_upper','pc_lower','pc_sd'))

In [13]:
#create 1000 draws of risks
risk_grid = risk_grid[, paste0('pc_',0:999) := lapply(0:999, function(x) rnorm(nrow(risk_grid) ,mean = pc_mean, sd = pc_sd))]
risk_grid = risk_grid[, version:= 'test']

In [15]:
#save R object to be used later
save(risk_grid, file = '/share/geospatial/temperature/estimates/risk/temperature_risks_test.Rdata')